In [1]:
from flair.data import TaggedCorpus, Sentence
from flair.data_fetcher import NLPTaskDataFetcher
import json
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, CharacterEmbeddings, FlairEmbeddings
from typing import List
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
import emoji

In [3]:
columns = {0: 'text', 1: 'ner'}

In [4]:
data_folder = "../data"

In [5]:
corpus: TaggedCorpus = NLPTaskDataFetcher.load_column_corpus(
    data_folder, columns,
    train_file='train.txt',
    dev_file='dev.txt',
    test_file='test.txt'
)

2019-05-11 17:23:04,339 Reading data from ../data
2019-05-11 17:23:04,340 Train: ../data/train.txt
2019-05-11 17:23:04,340 Dev: ../data/dev.txt
2019-05-11 17:23:04,341 Test: ../data/test.txt


In [6]:
corpus = corpus.downsample(0.50)

In [7]:
tag_dict = corpus.make_tag_dictionary(tag_type='ner')

In [8]:
len(corpus.train), len(corpus.dev), len(corpus.test)

(74364, 18592, 23240)

In [9]:
len(tag_dict.idx2item)

53

In [10]:
!head ../data/dev.txt

<START> O
What O
was O
the O
IRS O
thinking :thinking_face:
<STOP> O

<START> O
My O


In [11]:
tag_dict.idx2item

[b'<unk>',
 b'O',
 b':loudly_crying_face:',
 b':face_with_tears_of_joy:',
 b':weary_face:',
 b':double_exclamation_mark:',
 b':smiling_face_with_heart-eyes:',
 b':fire:',
 b':pleading_face:',
 b':female_sign:',
 b':skull:',
 b':folded_hands:',
 b':smiling_face_with_sunglasses:',
 b':person_shrugging:',
 b':male_sign:',
 b':winking_face:',
 b':person_facepalming:',
 b':thinking_face:',
 b':rolling_on_the_floor_laughing:',
 b':red_heart:',
 b':sparkles:',
 b':yellow_heart:',
 b':pensive_face:',
 b':blue_heart:',
 b':OK_hand:',
 b':eyes:',
 b':beaming_face_with_smiling_eyes:',
 b':heart_suit:',
 b':flexed_biceps:',
 b':speaking_head:',
 b':backhand_index_pointing_down:',
 b':face_blowing_a_kiss:',
 b':hugging_face:',
 b':sparkling_heart:',
 b':flushed_face:',
 b':two_hearts:',
 b':hundred_points:',
 b':purple_heart:',
 b':thumbs_up:',
 b':crying_face:',
 b':raising_hands:',
 b':face_with_rolling_eyes:',
 b':smiling_face:',
 b':grinning_face_with_sweat:',
 b':smiling_face_with_smiling_eyes

In [10]:
embedding_types: List[TokenEmbeddings] = [

    WordEmbeddings('twitter'),

    # comment in this line to use character embeddings
    #CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

2019-05-11 17:24:56,739 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/twitter.gensim.vectors.npy not found in cache, downloading to /tmp/tmpd7wjp8v6


100%|██████████| 477405728/477405728 [03:38<00:00, 2187037.03B/s]

2019-05-11 17:28:35,363 copying /tmp/tmpd7wjp8v6 to cache at /root/.flair/embeddings/twitter.gensim.vectors.npy


2019-05-11 17:28:36,408 removing temp file /tmp/tmpd7wjp8v6
2019-05-11 17:28:36,665 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/twitter.gensim not found in cache, downloading to /tmp/tmphjhu4zg5


100%|██████████| 68268001/68268001 [00:31<00:00, 2155522.95B/s]

2019-05-11 17:29:08,663 copying /tmp/tmphjhu4zg5 to cache at /root/.flair/embeddings/twitter.gensim


2019-05-11 17:29:08,798 removing temp file /tmp/tmphjhu4zg5


In [12]:
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

In [13]:
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dict,
                                        tag_type='ner',
                                        use_crf=True)

In [14]:
trainer: ModelTrainer = ModelTrainer(tagger, corpus)

In [ ]:
trainer.train('../models/flair-256-twitter',
              learning_rate=0.1,
              mini_batch_size=64,
              max_epochs=150, embeddings_in_memory=False)

2019-05-11 17:40:48,415 ----------------------------------------------------------------------------------------------------
2019-05-11 17:40:48,416 Evaluation method: MICRO_F1_SCORE
2019-05-11 17:40:48,418 ----------------------------------------------------------------------------------------------------
2019-05-11 17:40:49,755 epoch 1 - iter 0/1162 - loss 6.24337387
2019-05-11 17:43:00,905 epoch 1 - iter 116/1162 - loss 4.96789234
2019-05-11 17:45:10,293 epoch 1 - iter 232/1162 - loss 4.89085125
2019-05-11 17:47:19,917 epoch 1 - iter 348/1162 - loss 4.82975191
2019-05-11 17:49:30,152 epoch 1 - iter 464/1162 - loss 4.79444941
2019-05-11 17:51:40,075 epoch 1 - iter 580/1162 - loss 4.75228177
2019-05-11 17:54:01,303 epoch 1 - iter 696/1162 - loss 4.73039353
2019-05-11 17:56:11,471 epoch 1 - iter 812/1162 - loss 4.71208570
2019-05-11 17:58:20,423 epoch 1 - iter 928/1162 - loss 4.69834707
2019-05-11 18:00:33,120 epoch 1 - iter 1044/1162 - loss 4.67760613
2019-05-11 18:02:44,773 epoch 1 -

In [20]:
sentence = Sentence('Alternatively, try using a stacked embedding with FlairEmbeddings and GloVe, over the full data, for 150 epochs.')

In [24]:
sentence_emoji = Sentence("I love the conversation with a lot of emoji")

In [21]:
sentence_bert = Sentence("All of the results in the paper can be replicated in at most 1 hour on a single Cloud TPU, or a few hours on a GPU, starting from the exact same pre-trained model")

In [22]:
model = SequenceTagger.load_from_file('../models/flair-256/best-model.pt')

2019-05-11 17:10:00,468 loading file ../models/flair-256/best-model.pt


In [23]:
model.predict(sentences=[sentence, sentence_bert])
print(sentence.to_tagged_string())
print(sentence_bert.to_tagged_string())

Alternatively, try using a stacked embedding with FlairEmbeddings and GloVe, over the full data, for 150 epochs.
All of the results in the paper can be replicated in at most 1 hour on a single Cloud TPU, or a few hours on a GPU, starting from the exact same pre-trained model


In [39]:
!head -n 26 ../../TwitterLM/tweets.txt

💜 Hello 
So yesterday I got my self a new car. God is awesome! 😭😭 
 Lord bustta I greet you 🙌🏿🙌🏿
ADELIN MADE ME YOUR FRIEND 😔
it’s already 2019 but my heart still belongs to namlee ☹️💖 
Rt or have bad luck for the rest of the school year 🤦🏻‍♂️ 
Idris Elba marries Sabrina Dhowre in beautiful Moroccan wedding 😍 
I’m still laughing at Hero’s reaction😁 
190427 magical circus Kobe 🥰❤️🥰❤️🥰❤️🥰❤️🥰❤️😘 
Little date with my handsome ❤️ 
I'm crying for this guy rn 👇🏽😥🤦🏽😂😂🤣🤣 
ali, while i'm beside my brother: how was last night? nalasing ka ba? ☹️☹️☹️
the way i cry over this man 😭💜 
Good night... Sad 😭 
Hello I'm Bea nice to meet you 💗 SSFW EXO 
😎LAST CHANCE TO BOOK: Half Price Flash Sale Exclusive - Mexico ✈ - 
Yes, but the men in that movie are just 😍😍😍😍
Imagine having this much of an impact on the planet, incredible 💛 
 Sure we will just have our own party 👍
Y’all ever notice how Yoonji and Yoongi kinda look alike 🤔 BTS 
A man who plays for the badge. 🇰🇪
How is this true around the world? 😂 Does

In [64]:
emoji.emojize(model.predict(Sentence(" Idris Elba marries Sabrina Dhowre in beautiful Moroccan wedding doesn't it?"))[0].to_tagged_string())

"Idris Elba marries Sabrina Dhowre in beautiful Moroccan wedding <😍> doesn't it?"

In [26]:
sentence_emoji.to_tagged_string()

'I love the conversation with a lot of emoji'

In [63]:
from flair.visual.training_curves import Plotter

In [65]:
plotter = Plotter()
plotter.plot_training_curves('../models/flair-testing/loss.tsv')
#plotter.plot_weights('../models/flair-testing/weights.txt')